In [25]:
# **IMPORTANDO LIBRERIAS**

In [26]:
import sys
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy import create_engine, Column, Integer, String, VARCHAR,Boolean, TIMESTAMP
from sqlalchemy.types import DateTime
from sqlalchemy.orm import sessionmaker


sys.path.append(os.path.abspath(os.path.join('..')))

In [27]:
load_dotenv()  # Carga las variables del archivo .env

config = {
    'host': os.getenv('HOST'),
    'user': os.getenv('USER'),
    'password': os.getenv('PASSWORD'),
    'database': os.getenv('DATABASE'),
    'port': os.getenv('PORT')
}

In [28]:
# Crear la URL de conexión para SQLAlchemy
db_url = f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/caso_7"

# Crear el motor de SQLAlchemy
engine = create_engine(db_url, echo=True)



In [29]:
Base = declarative_base()


# **1. GENERACIÓN DE TABLAS**

In [30]:
class ProductHierarchy(Base):

    __tablename__ = 'product_hierarchy'
    __table_args__= {'extend_existing':True}

    #| ------------------ Columnas ----------------|

    id =Column(Integer, primary_key=True)
    parent_id = Column(Integer)
    level_text = Column(VARCHAR(19))
    level_name = Column(VARCHAR(8))


class ProductPrices(Base):

    __tablename__ = 'product_prices'
    __table_args__= {'extend_existing':True}

    #| ------------------ Columnas ----------------|
    
    id =Column(Integer, primary_key=True)
    product_id = Column(VARCHAR(6))
    price = Integer

class ProductDetails(Base):

    __tablename__ = 'product_details'
    __table_args__= {'extend_existing':True}
    
    #| ------------------ Columnas ----------------|

    product_id = Column(VARCHAR(6), primary_key=True)
    price = Column(Integer)
    product_name = Column(VARCHAR(32))
    category_id = Column(Integer)
    segment_id = Column(Integer)
    style_id = Column(Integer)
    category_name = Column(VARCHAR(6))
    segment_name =Column(VARCHAR(6))
    style_name = Column(VARCHAR(19))


class Sales(Base):

    __tablename__ = 'sales'
    __table_args__= {'extend_existing':True}

 #| ------------------ Columnas ----------------|
    id = Column(Integer, primary_key=True)
    prod_id = Column(VARCHAR(6))
    price = Column(Integer)
    discount = Column(Integer)
    member = Column(Boolean)
    txn_id =Column(VARCHAR(32))
    start_txn_time = Column(TIMESTAMP(timezone=True))

Base.metadata.create_all(engine)


2025-04-28 14:21:24,432 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-04-28 14:21:24,432 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-28 14:21:24,434 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-04-28 14:21:24,435 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-28 14:21:24,435 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-04-28 14:21:24,436 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-28 14:21:24,438 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-28 14:21:24,438 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`product_hierarchy`


2025-04-28 14:21:24,439 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-28 14:21:24,445 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`product_prices`
2025-04-28 14:21:24,446 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-28 14:21:24,448 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`product_details`
2025-04-28 14:21:24,449 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-28 14:21:24,450 INFO sqlalchemy.engine.Engine DESCRIBE `caso_7`.`sales`
2025-04-28 14:21:24,452 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-28 14:21:24,454 INFO sqlalchemy.engine.Engine 
CREATE TABLE product_details (
	product_id VARCHAR(6) NOT NULL, 
	price INTEGER, 
	product_name VARCHAR(32), 
	category_id INTEGER, 
	segment_id INTEGER, 
	style_id INTEGER, 
	category_name VARCHAR(6), 
	segment_name VARCHAR(6), 
	style_name VARCHAR(19), 
	PRIMARY KEY (product_id)
)


2025-04-28 14:21:24,455 INFO sqlalchemy.engine.Engine [no key 0.00132s] {}
2025-04-28 14:21:24,473 INFO sqlalchemy.engine.Engine 
CR

## **1.1 CARGANDO LOS DATOS**

In [31]:
from utils import funciones_utiles

product_hierarchy_data = funciones_utiles.load_text_as_tuples('datos_tablas/product_hierarchy_data.txt')
product_prices_data = funciones_utiles.load_text_as_tuples('datos_tablas/product_prices_data.txt')
product_details_data =funciones_utiles.load_text_as_tuples('datos_tablas/product_details_data.txt')
sales_data = funciones_utiles.load_text_as_tuples('datos_tablas/sales_data.txt')


In [32]:
# Creando la session
Session = sessionmaker(bind=engine)
session = Session()

In [33]:
product_hierarchy_data

[('1', None, 'Womens', 'Category'),
 ('2', None, 'Mens', 'Category'),
 ('3', '1', 'Jeans', 'Segment'),
 ('4', '1', 'Jacket', 'Segment'),
 ('5', '2', 'Shirt', 'Segment'),
 ('6', '2', 'Socks', 'Segment'),
 ('7', '3', 'Navy Oversized', 'Style'),
 ('8', '3', 'Black Straight', 'Style'),
 ('9', '3', 'Cream Relaxed', 'Style'),
 ('10', '4', 'Khaki Suit', 'Style'),
 ('11', '4', 'Indigo Rain', 'Style'),
 ('12', '4', 'Grey Fashion', 'Style'),
 ('13', '5', 'White Tee', 'Style'),
 ('14', '5', 'Teal Button Up', 'Style'),
 ('15', '5', 'Blue Polo', 'Style'),
 ('16', '6', 'Navy Solid', 'Style'),
 ('17', '6', 'White Striped', 'Style'),
 ('18', '6', 'Pink Fluro Polkadot', 'Style')]

### **1.1.1 Cargando datos `product_hierarchy`**

In [35]:

objects = [
    ProductHierarchy(

        id =int(id),
        parent_id = int(parent_id) if parent_id is not None else None,
        level_text = level_text,
        level_name = level_name
    )
    for id, parent_id, level_text, level_name in product_hierarchy_data
]

session.bulk_save_objects(objects)
session.commit()

2025-04-28 14:21:36,857 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-28 14:21:36,858 INFO sqlalchemy.engine.Engine INSERT INTO product_hierarchy (id, level_text, level_name) VALUES (%(id)s, %(level_text)s, %(level_name)s)
2025-04-28 14:21:36,859 INFO sqlalchemy.engine.Engine [generated in 0.00081s] [{'id': 1, 'level_text': 'Womens', 'level_name': 'Category'}, {'id': 2, 'level_text': 'Mens', 'level_name': 'Category'}]
2025-04-28 14:21:36,862 INFO sqlalchemy.engine.Engine INSERT INTO product_hierarchy (id, parent_id, level_text, level_name) VALUES (%(id)s, %(parent_id)s, %(level_text)s, %(level_name)s)
2025-04-28 14:21:36,862 INFO sqlalchemy.engine.Engine [generated in 0.00070s] [{'id': 3, 'parent_id': 1, 'level_text': 'Jeans', 'level_name': 'Segment'}, {'id': 4, 'parent_id': 1, 'level_text': 'Jacket', 'level_name': 'Segment'}, {'id': 5, 'parent_id': 2, 'level_text': 'Shirt', 'level_name': 'Segment'}, {'id': 6, 'parent_id': 2, 'level_text': 'Socks', 'level_name': 'Segment'}, {'

### **1.1.2 Cargando datos `ProductPrices`**

In [36]:

objects = [
    ProductPrices(

        id =int(id),
        product_id = product_id,
        price = int(price)
    )
    for id, product_id, price in product_prices_data
]

session.bulk_save_objects(objects)
session.commit()

2025-04-28 14:21:39,657 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-28 14:21:39,658 INFO sqlalchemy.engine.Engine INSERT INTO product_prices (id, product_id) VALUES (%(id)s, %(product_id)s)
2025-04-28 14:21:39,659 INFO sqlalchemy.engine.Engine [generated in 0.00073s] [{'id': 7, 'product_id': 'c4a632'}, {'id': 8, 'product_id': 'e83aa3'}, {'id': 9, 'product_id': 'e31d39'}, {'id': 10, 'product_id': 'd5e9a6'}, {'id': 11, 'product_id': '72f5d4'}, {'id': 12, 'product_id': '9ec847'}, {'id': 13, 'product_id': '5d267b'}, {'id': 14, 'product_id': 'c8d436'}  ... displaying 10 of 12 total bound parameter sets ...  {'id': 17, 'product_id': 'b9a74d'}, {'id': 18, 'product_id': '2feb6b'}]
2025-04-28 14:21:39,661 INFO sqlalchemy.engine.Engine COMMIT


### **1.1.3 Cargando datos `ProductDetails`**

In [37]:
objects = [
    ProductDetails(

        product_id = product_id, 
        price = int(price),
        product_name = product_name,
        category_id = int(category_id),
        segment_id = int(segment_id),
        style_id = int(style_id),
        category_name = category_name,
        segment_name =segment_name,
        style_name = style_name,
    )
    for product_id, price, product_name, category_id, segment_id,  style_id, category_name,  segment_name, style_name in product_details_data
]

session.bulk_save_objects(objects)
session.commit()

2025-04-28 14:21:42,980 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-28 14:21:42,983 INFO sqlalchemy.engine.Engine INSERT INTO product_details (product_id, price, product_name, category_id, segment_id, style_id, category_name, segment_name, style_name) VALUES (%(product_id)s, %(price)s, %(product_name)s, %(category_id)s, %(segment_id)s, %(style_id)s, %(category_name)s, %(segment_name)s, %(style_name)s)
2025-04-28 14:21:42,983 INFO sqlalchemy.engine.Engine [generated in 0.00081s] [{'product_id': 'c4a632', 'price': 13, 'product_name': 'Navy Oversized Jeans - Womens', 'category_id': 1, 'segment_id': 3, 'style_id': 7, 'category_name': 'Womens', 'segment_name': 'Jeans', 'style_name': 'Navy Oversized'}, {'product_id': 'e83aa3', 'price': 32, 'product_name': 'Black Straight Jeans - Womens', 'category_id': 1, 'segment_id': 3, 'style_id': 8, 'category_name': 'Womens', 'segment_name': 'Jeans', 'style_name': 'Black Straight'}, {'product_id': 'e31d39', 'price': 10, 'product_name': 'Cream 